#### Задача 1. 

Напишите метакласс с логированием: при определении любого нового класса в файл (путь к которому хранится в статическом атрибуте метакласса) записывается имя этого класса, его суперклассы и атрибуты. 

In [1]:
class Meta(type):
    path = r'D:\\uni\\new.txt'
    def __new__(cls, name, bases, dct):
        x = super().__new__(cls, name, bases, dct)
        with open(cls.path, 'a', encoding='utf8') as file:
            for key, value in dct.items():
                file.write(f'{key}: {value}')
        return x    
    

class Child(metaclass=Meta):
    def __init__(self, name):
        self.name = name




#### Задача 2. 

Напишите метакласс, который любому новому классу будет добавлять метод sayHello(\*args, \*\*kwargs). 

In [56]:
def say_hello(self, string):
        print(f'Hello, {string}')
        
class Meta(type):
     def __new__(cls, name, bases, dct):
        x = super().__new__(cls, name, bases, dct)
        x.hello = classmethod(say_hello)
        return x
        

class Child(metaclass=Meta): 
    def __init__(self, name):
        self.name = name


c = Child('fufu')
c.hello(c.name)

Hello, fufu


In [62]:
vars(c)

{'name': 'fufu'}

In [59]:

        
class Meta(type):
     def __new__(cls, name, bases, dct):
        def say_hello(*args, **kwargs):
            print(f'Hello')
        x = super().__new__(cls, name, bases, dct)
        x.hello = say_hello
        return x
        

class Child(metaclass=Meta): 
    def __init__(self, name):
        self.name = name


c = Child('fufu')
c.hello()

Hello


#### Задача 3. 

Напишите метакласс, который позволяет своим классам иметь только атрибуты name, surname, age и никаких других. 

In [13]:
class Meta(type):
    def __new__(cls, name, bases, dic):
        dic['__slots__'] = ('name', 'surname', 'age')
        return super().__new__(cls, name, bases, dic)



In [ ]:
class Meta(type):
    def __new__(cls, name, bases, dic):
        dic['__slots__'] = ('name', 'surname', 'age')
        return super().__new__(cls, name, bases, dic)
    
class A(metaclass=Meta): pass 

dir(A)

In [19]:
class A:
    __slots__ = ('name',)

a = A()
a.name = 1
a.surname = 1

AttributeError: 'A' object has no attribute 'surname'

In [14]:
class Named(metaclass= Meta):
    def __init__(self, name, surname, age):
        self.name = name
        self.surname = surname
        self.age = age

c = Named('dv', 'asdcf', 12)

In [15]:
c.map = 1

AttributeError: 'Named' object has no attribute 'map'

In [12]:
class Named(metaclass= Meta):
    def __init__(self, name, surname, age):
        self.name = name
        self.notsurname = surname
        self.age = age

c = Named('dv', 'asdcf', 12)

ValueError: Excess of parameters.

#### Задача 4 (2 балла). 

Приготовимся в большом проекте создать игру-стратегию (графику потом дизайнеры добавят). Наша игра будет называться Dawn of Man, и задачей игрока будет управлять поселением первобытных людей, которые в процессе набирают очки знаний и открывают новые технологии. Наши люди - это отдельные юниты, которые умеют взаимодействовать с элементами окружающей среды: у них будут какие-то инструменты для создания еды, оружия и прочего (например, какой-нибудь навес, под которым человек садится, набрав палок и камней, и ваяет себе палку-копалку). 

Наша задача - набросать черновик для игры: создать абстрактные классы для людей, объектов, с которыми они взаимодействуют, чтобы создавать вещи, и собственно вещей. Возможно, имеет смысл также предусмотреть черновик-класс для животных, на которых люди будут охотиться. 

Попробуйте также создать конкретный класс для кого-нибудь из них. 

In [ ]:
from abc import ABC, abstractmethod
from random import randint, choice

class Human(ABC):
    def __init__(self, name, gender, age : int):
        self.name = name
        self.age = age
        self.gender = gender
        self.wisdom = 0
        self.energy = 10
        self.supply = {}
        self.friends = {}
        self.mood = 10
    
    @abstractmethod
    def sleep(self, n):
        self.energy += n
        print(f'{self.name} спит {n} часов и восстанавливает энергию до {self.energy}')
    
    @abstractmethod
    def eat(self, food):
        if food.name in self.supply:
            self.energy += food.energy
            self.supply[food.name] -= 1
            print(f'{self.name} съедает {food.name}, восстанавливает энергию до {self.energy}')
        else:
            print(f'{food.name} нет в запасах!')
    
    @abstractmethod
    def explore(self, region):
        found_item = Instrument("Примитивное оружие")
        self.wisdom += 1
        print(f'{self.name} исследует {region}, повышает мудрость до {self.wisdom} и находит {found_item.name}')
    
    @abstractmethod
    def get_food(self, instrument):
        if instrument.category == "weapon":
            animal_size = randint(10, 20) if instrument.level >= 4 else randint(5, 10) if instrument.level == 3 else randint(1, 5)
            hunted_animal = Animal(species="Животное", size=animal_size) #можно сделать класс для мира, в котором будут храниться все доступные виды животных и выбирать из них, но у меня не хватило терпения
            print(f'{self.name} охотится с {instrument.name} и добывает {hunted_animal.size} животное')
            meat = Food(name="мясо", energy=5)
            meat.set_category("meat")
            
            if meat.name in self.supply:
                self.supply[meat.name] += hunted_animal.size
            else:
                self.supply[meat.name] = hunted_animal.size
            print(f'Добавлено мясо. Теперь в запасе: {self.supply[meat.name]} единиц мяса')

        elif instrument.category == "instrument":
            plant = Food(name="Овощ", energy=3)
            plant.set_category("plant")
            amount = instrument.level

            if plant.name in self.supply:
                self.supply[plant.name] += amount
            else:
                self.supply[plant.name] = amount
            print(f'{self.name} собирает растения с помощью {instrument.name}. Теперь в запасе: {self.supply[plant.name]} единиц растений')
    
    @abstractmethod
    def communicate(self, human):
        if human not in self.friends:
            self.friends[human] = 0  
        
        mood_effect = choice([-1, 1]) if (self.mood + human.mood) > 10 else choice([-1, -1, 1]) #в зависимости от того, какое у обоих настроение, они либо поругаются, либо вместе порадуются, правда в любом случае станут друзьямм
        
        if mood_effect == 1:
            self.mood += randint(1, 3)
            self.friends[human] += 1
            print(f'{self.name} и {human.name} сегодня друг другу нравятся, настроение увеличилось до {self.mood}, отношения {self.friends[human]}')
        else:
            self.mood -= randint(1, 3)
            self.friends[human] -= 1
            print(f'{self.name} и {human.name} сегодня друг друга раздражают, настроение снизилось до {self.mood}, отношения {self.friends[human]}')


class Food(ABC):
    def __init__(self, name, energy: int):
        self.name = name
        self.energy = energy
    
    @abstractmethod
    def get_category(self):
        return self.category

    @abstractmethod
    def set_category(self, value):
        if value in ['meat', 'plant']:
            self.category = value
    category = property(get_category, set_category)



class Instrument(ABC):
    def __init__(self, name):
        self.name = name
        self.level = randint(1, 5)
    
    @abstractmethod
    def get_category(self):
        return self.category

    @abstractmethod
    def set_category(self, value):
        if value in ['instrument', 'weapon']:
            self.category = value
    category = property(get_category, set_category)



class Animal(ABC):
    def __init__(self, species, size : int):
        self.species = species
        self.size = size

    


class PrimitiveHuman(Human):
    def sleep(self, n):
        super().sleep(n) 
    
    def eat(self, food):
        super().eat(food)
    
    def explore(self, region):
        super().explore(region)
    
    def get_food(self, instrument):
        super().get_food(instrument)
    
    def communicate(self, human):
        super().communicate(human)


h = PrimitiveHuman('Petya', 'male', 20) 
s = PrimitiveHuman('Masha', 'female', 18) 
h.communicate(s)
   
#не уверена, что до конца поняла, что было нужно сделать, как будто бы получилось, что абстрактные классы в моей задаче вполне могли быть конкретными и ничего не поменялось бы...


    

Petya и Masha сегодня друг другу нравятся, настроение увеличилось до 13, отношения 1
